In [1]:
import findspark


In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession

In [4]:
sc = SparkSession.builder.appName('broadcast_accumulators').config('spark.driver.host','localhost').getOrCreate()

In [5]:
spark = sc.sparkContext

In [6]:
spark

<SparkContext master=local[*] appName=broadcast_accumulators>

WHAT IS THE DIFFERENCE BETWEEN SPARKSESSION AND SPARKCONTEXT

sparkSession: Introduced in spark2.0 it is a unified entry point to work with spark. Encapsulates the sparkContext ,sqlContext and HiveContext
a) it has functionalities to work with DataFrames , DataSets and SQLAPIs

sparkContext: focuses mainly on low level APIs for RDD
    * USED FOR OPERATIONS SUCH AS CREATING RDD AND MANAGING CLUSTER RESOURCES

In [7]:
rdd = spark.parallelize([1,2,4,5,5])
# sparkContext is used to make RDDS

In [8]:
rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [9]:
rdd.collect()

[1, 2, 4, 5, 5]

In [10]:
df = sc.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])

In [11]:
df

DataFrame[id: bigint, name: string]

In [12]:
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



# broadcast and accumulators

Broadcast: read-only large file/dataset that is distributed /broadcasted accross the nodes 

In [13]:
large_lookup_table ={"apple":"fruit","banana":"fruit","maruti":"car","broccolli":"vegetable","chicken":"meat"}

In [14]:
type(large_lookup_table)

dict

In [15]:
broadcast_var = spark.broadcast(large_lookup_table)

In [16]:
def classify(word):
    lookup_value = broadcast_var.value.get(word,"unknown")
    return lookup_value

In [17]:
thing_list=["apple","banana","ball","car"]
rdd_list = spark.parallelize(thing_list)
classification = rdd_list.map(classify)


In [18]:
print(classification.collect())

['fruit', 'fruit', 'unknown', 'unknown']


In [19]:
broadcast_var.value.get('apple')

'fruit'

ACCUMULATORS

Accumulators are shared , write-only variables that are used for aggregations of values accross tasks in a distributed environment.
* They are primarily used to track global metrics or counters in a spark applications
* while tasks can only update the value of accumulators , driver program can read it

In [20]:
sample_data = [1,2,3,4,5,6,7]
rdd = spark.parallelize(sample_data)

# defince accumulator
accum = spark.accumulator(0)
rdd.foreach(lambda x: accum.add(x))


In [21]:
print(accum.value)

28
